In [1]:

import pandas as pd
import os # Para verificar a existência de arquivos e manipular caminhos

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [3]:
import os
import re
from pathlib import Path

def quebrar_txt_por_id(caminho_arquivo_entrada, diretorio_saida, separador_id=","):
    """
    Lê um arquivo .txt (ou .csv) e o divide em múltiplos arquivos com base em um ID.

    Cada novo arquivo conterá todas as linhas que compartilham o mesmo ID.
    O ID é extraído da primeira parte de cada linha, usando um separador.
    Ao escrever as linhas no arquivo de saída, o primeiro campo (ID) é removido.

    Nome do arquivo de saída: <nome_origem_sem_extensão><separador_id><ID>.txt
    Ex.: origem = "arquivo.csv", separador_id="," e ID="ABC" => "arquivo,ABC.txt"

    Args:
        caminho_arquivo_entrada (str): Caminho completo para o arquivo de entrada (.txt/.csv).
        diretorio_saida (str): Diretório onde os novos arquivos serão salvos.
        separador_id (str): Caractere usado para separar o ID do restante da linha (ex: ';', ',', ' ').
                            Se não houver separador, a linha inteira será considerada ID e não haverá remoção de campo.
    """

    # Garante o diretório de saída
    Path(diretorio_saida).mkdir(parents=True, exist_ok=True)

    # Dicionário para armazenar as linhas (sem o primeiro campo) por ID
    linhas_por_id = {}

    # Nome base do arquivo de entrada (sem extensão)
    nome_origem = Path(caminho_arquivo_entrada).stem

    # Função para sanitizar ID para uso em nome de arquivo
    def sanitizar_id(id_texto):
        # Remove caracteres não seguros para nomes de arquivos
        id_limpo = re.sub(r'[\\/:*?"<>|]+', '_', id_texto.strip())
        # Limita tamanho opcionalmente (evita nomes gigantes)
        return id_limpo[:150] if len(id_limpo) > 150 else id_limpo

    try:
        with open(caminho_arquivo_entrada, 'r', encoding='utf-8') as f_entrada:
            for numero_linha, linha in enumerate(f_entrada, 1):
                linha = linha.rstrip('\n').strip()

                if not linha:
                    continue  # pula linhas vazias

                # Extrai ID e conteúdo (sem o primeiro campo)
                if separador_id and (separador_id in linha):
                    id_atual, resto = linha.split(separador_id, 1)  # ID e o restante da linha
                    id_atual = id_atual.strip()
                    conteudo_sem_id = resto.strip()
                else:
                    # Sem separador: toda a linha é o ID e não há conteúdo sem ID
                    id_atual = linha.strip()
                    conteudo_sem_id = ""  # ou poderíamos manter a linha, mas você pediu excluir o primeiro campo

                    print(f"Aviso: Separador '{separador_id}' não encontrado na linha {numero_linha}. "
                          f"Usando a linha inteira como ID: '{id_atual}'. Nenhum conteúdo para escrever (somente ID).")

                if not id_atual:
                    print(f"Aviso: ID vazio na linha {numero_linha}. Linha ignorada.")
                    continue

                id_arquivo = sanitizar_id(id_atual)

                if id_arquivo not in linhas_por_id:
                    linhas_por_id[id_arquivo] = []

                # Adiciona somente o conteúdo sem o campo de ID
                if conteudo_sem_id != "":
                    linhas_por_id[id_arquivo].append(conteudo_sem_id)

        # Escreve os arquivos de saída
        total_arquivos = 0
        for id_gerado, linhas_do_id in linhas_por_id.items():
            # Monta o nome do arquivo: origem + separador_id + ID + .txt
            nome_arquivo_saida = f"{nome_origem}{separador_id}{id_gerado}.txt"
            caminho_saida = Path(diretorio_saida) / nome_arquivo_saida

            with open(caminho_saida, 'w', encoding='utf-8') as f_saida:
                for conteudo in linhas_do_id:
                    f_saida.write(conteudo + '\n')

            total_arquivos += 1
            print(f"Arquivo '{caminho_saida}' criado com {len(linhas_do_id)} linha(s).")

        print(f"\nProcesso concluído! {total_arquivos} arquivo(s) foram gerados em '{diretorio_saida}'.")

    except FileNotFoundError:
        print(f"Erro: O arquivo de entrada '{caminho_arquivo_entrada}' não foi encontrado.")
    except PermissionError:
        print(f"Erro: Sem permissão para ler/escrever nos caminhos informados.")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


# --- CONFIGURAÇÃO ---
# Caminho para o seu arquivo de entrada
arquivo_entrada = r"C:/Users/aline.chaves/OneDrive - pluxeegroup/Documentos Compartilhados - Digital - Disparos de base/Push - Consumer/20251119_SDESKBRA-247385.csv"

# Diretório onde os arquivos menores serão salvos
diretorio_saida = r"C:/Users/aline.chaves/OneDrive - pluxeegroup/Documentos Compartilhados - Digital - Disparos de base/Push - Consumer/"

# O separador que antecede o ID na sua linha (ex: "ID123;dado1;dado2" -> separador é ",")
separador = ","
# --- FIM CONFIGURAÇÃO ---

if __name__ == "__main__":
    quebrar_txt_por_id(arquivo_entrada, diretorio_saida, separador)

Arquivo 'C:\Users\aline.chaves\OneDrive - pluxeegroup\Documentos Compartilhados - Digital - Disparos de base\Push - Consumer\20251119_SDESKBRA-247385,﻿id_merchant_campanha.txt' criado com 1 linha(s).
Arquivo 'C:\Users\aline.chaves\OneDrive - pluxeegroup\Documentos Compartilhados - Digital - Disparos de base\Push - Consumer\20251119_SDESKBRA-247385,3646597.txt' criado com 203065 linha(s).
Arquivo 'C:\Users\aline.chaves\OneDrive - pluxeegroup\Documentos Compartilhados - Digital - Disparos de base\Push - Consumer\20251119_SDESKBRA-247385,3552993.txt' criado com 203065 linha(s).
Arquivo 'C:\Users\aline.chaves\OneDrive - pluxeegroup\Documentos Compartilhados - Digital - Disparos de base\Push - Consumer\20251119_SDESKBRA-247385,3553028.txt' criado com 203065 linha(s).
Arquivo 'C:\Users\aline.chaves\OneDrive - pluxeegroup\Documentos Compartilhados - Digital - Disparos de base\Push - Consumer\20251119_SDESKBRA-247385,3059649.txt' criado com 203065 linha(s).
Arquivo 'C:\Users\aline.chaves\OneDr

'C:/Users/aline.chaves/OneDrive - pluxeegroup/Documentos Compartilhados - Digital - Disparos de base/Push - Consumer/20250929_SDESKBRA-226654_leve_mais'